In [467]:
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import warnings
warnings.filterwarnings('ignore')

In [269]:
#get law groups
driver = webdriver.Chrome(executable_path='/mnt/c/charin_projects/thai-law/chromedriver.exe')
driver.get('https://www.krisdika.go.th/web/guest/thai-code-annotated')
soup = BeautifulSoup(driver.page_source)
driver.close()

law_groups = pd.DataFrame([i.text for i in soup.find_all('a',class_='ksdk-theme-bg-third-color')])
law_groups['law_group'] = law_groups[0].map(lambda x: x.split('(')[0][:-1])
law_groups['nb_laws'] = law_groups[0].map(lambda x: int(x.split('(')[1][:-1]))
law_groups['nb_pages'] = np.ceil(law_groups['nb_laws']/10).astype(int)
law_groups = law_groups.drop(0,1)
# law_groups.to_csv('data/v0.3/law_groups.csv',index=False)
law_groups

,law_group,nb_laws,nb_pages
0,กษัตริย์,12,2
1,การเงิน การคลัง และวิธีการงบประมาณ,146,15
2,การเมืองการปกครอง,269,27
3,ขนส่งและคมนาคม,59,6
4,คณะกรรมการกฤษฏีกา,2,1
5,คนต่างด้าว,56,6
6,ครอบครัว และมรดก,69,7
7,ความมั่นคง และการรักษาความสงบเรียบร้อย,53,6
8,ความเท่าเทียมระหว่างเพศ,1,1
9,คุ้มครองผู้บริโภค,56,6


In [270]:
def get_law_urls(law_group, nb_pages):
    #open list page
    driver = webdriver.Chrome(executable_path='chromedriver.exe')
    driver.get('https://www.krisdika.go.th/web/guest/thai-code-annotated')
    #click law group
    link = driver.find_element_by_partial_link_text(law_group)
    link.click()
    
    #check if max pagination button appeared
    try:
        element = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.LINK_TEXT, str(nb_pages)))
        )
    except:
        print('Max pagination button not found')
    
    #get law _urls
    laws = []
    law_urls = []
    for nb_page in tqdm(range(1, nb_pages+1)):
        link = driver.find_element_by_link_text(str(nb_page))
        link.click()
        soup = BeautifulSoup(driver.page_source)
        laws+=[i.text for i in soup.find_all('li', class_='thca-list-law-name')]
        law_urls+=[i.find_all('li')[-1].find('a').get('href') for i in soup.find_all('ul', class_='thca-list-icon')]
        
    #close driver
    driver.close()
    
    #summarize to df
    df = pd.DataFrame({'title':laws, 'law_url':law_urls})
    df['sysid'] = df.law_url.map(lambda x: x.split('=')[-2].split('&')[0])
    df['law_group'] = law_group
    return df

In [272]:
dfs = []
for row in tqdm(law_groups.itertuples(index=False)):
    print(row[0])
    df = get_law_urls(row[0],row[2])
    dfs.append(df)

กษัตริย์



การเงิน การคลัง และวิธีการงบประมาณ



การเมืองการปกครอง



ขนส่งและคมนาคม



คณะกรรมการกฤษฏีกา


...


เครื่องราชอิสริยาภรณ์



เครื่องแบบ เสื้อครุย



เอกสิทธิ์และความคุ้มกัน



แรงงาน



โทรคมนาคม และการสื่อสาร


In [273]:
law_url_df = pd.concat(dfs)
law_url_df

,title,law_url,sysid,law_group
0,พระราชกำหนดโอนอัตรากำลังพลและงบประมาณบางส่วนขอ...,https://www.krisdika.go.th/librarian/get?sysid...,843218,กษัตริย์
1,พระราชบัญญัติระเบียบบริหารราชการในพระองค์ พ.ศ....,https://www.krisdika.go.th/librarian/get?sysid...,775999,กษัตริย์
2,รัฐธรรมนูญแห่งราชอาณาจักรไทย,https://www.krisdika.go.th/librarian/get?sysid...,774606,กษัตริย์
3,ประมวลกฎหมายอาญา (ฉบับ Update ล่าสุด),https://www.krisdika.go.th/librarian/get?sysid...,443287,กษัตริย์
4,ประมวลกฎหมายวิธีพิจารณาความอาญา (ฉบับ Update ล...,https://www.krisdika.go.th/librarian/get?sysid...,570066,กษัตริย์
...,...,...,...,...
7,พระราชบัญญัติจดแจ้งการพิมพ์ พ.ศ. 2550,https://www.krisdika.go.th/librarian/get?sysid...,568883,โทรคมนาคม และการสื่อสาร
8,พระราชบัญญัติคุ้มครองการดำเนินงานขององค์การโทร...,https://www.krisdika.go.th/librarian/get?sysid...,319040,โทรคมนาคม และการสื่อสาร
9,พระราชบัญญัติคุ้มครองการดำเนินงานขององค์การโทร...,https://www.krisdika.go.th/librarian/get?sysid...,302303,โทรคมนาคม และการสื่อสาร
10,พระราชบัญญัติกองทุนพัฒนาสื่อปลอดภัยและสร้างสรร...,https://www.krisdika.go.th/librarian/get?sysid...,725393,โทรคมนาคม และการสื่อสาร


In [323]:
#designate law types
law_types = ['รัฐธรรมนูญ', 'พระราชบัญญัติ', 'พระราชกำหนด','ประมวลกฎหมาย', 'ประมวลรัษฎากร','ประกาศ','คำสั่ง','พระธรรมนูญ']

def get_law_type(law_name, law_types=law_types, max_char=13):
    for l in law_types:
        if l in law_name[:max_char]:
            return l
    return 'others'

law_url_df['law_type'] = law_url_df.title.map(get_law_type)
law_url_df.law_type.value_counts()

พระราชบัญญัติ    2012
พระราชกำหนด        81
ประมวลกฎหมาย       54
รัฐธรรมนูญ         32
ประกาศ             21
ประมวลรัษฎากร      11
คำสั่ง              3
พระธรรมนูญ          1
Name: law_type, dtype: int64

In [327]:
law_url_df[['sysid','law_type']].drop_duplicates().law_type.value_counts()

พระราชบัญญัติ    800
พระราชกำหนด       43
ประกาศ            13
ประมวลกฎหมาย       6
คำสั่ง             3
ประมวลรัษฎากร      1
พระธรรมนูญ         1
รัฐธรรมนูญ         1
Name: law_type, dtype: int64

In [342]:
# law_url_df.to_csv('data/v0.3/law_url_df.csv',index=False)
law_url_df = pd.read_csv('data/v0.3/law_url_df.csv')
law_url_df

,title,law_url,sysid,law_group,law_type
0,พระราชกำหนดโอนอัตรากำลังพลและงบประมาณบางส่วนขอ...,https://www.krisdika.go.th/librarian/get?sysid...,843218,กษัตริย์,พระราชกำหนด
1,พระราชบัญญัติระเบียบบริหารราชการในพระองค์ พ.ศ....,https://www.krisdika.go.th/librarian/get?sysid...,775999,กษัตริย์,พระราชบัญญัติ
2,รัฐธรรมนูญแห่งราชอาณาจักรไทย,https://www.krisdika.go.th/librarian/get?sysid...,774606,กษัตริย์,รัฐธรรมนูญ
3,ประมวลกฎหมายอาญา (ฉบับ Update ล่าสุด),https://www.krisdika.go.th/librarian/get?sysid...,443287,กษัตริย์,ประมวลกฎหมาย
4,ประมวลกฎหมายวิธีพิจารณาความอาญา (ฉบับ Update ล...,https://www.krisdika.go.th/librarian/get?sysid...,570066,กษัตริย์,ประมวลกฎหมาย
...,...,...,...,...,...
2210,พระราชบัญญัติจดแจ้งการพิมพ์ พ.ศ. 2550,https://www.krisdika.go.th/librarian/get?sysid...,568883,โทรคมนาคม และการสื่อสาร,พระราชบัญญัติ
2211,พระราชบัญญัติคุ้มครองการดำเนินงานขององค์การโทร...,https://www.krisdika.go.th/librarian/get?sysid...,319040,โทรคมนาคม และการสื่อสาร,พระราชบัญญัติ
2212,พระราชบัญญัติคุ้มครองการดำเนินงานขององค์การโทร...,https://www.krisdika.go.th/librarian/get?sysid...,302303,โทรคมนาคม และการสื่อสาร,พระราชบัญญัติ
2213,พระราชบัญญัติกองทุนพัฒนาสื่อปลอดภัยและสร้างสรร...,https://www.krisdika.go.th/librarian/get?sysid...,725393,โทรคมนาคม และการสื่อสาร,พระราชบัญญัติ


In [343]:
law_groups.nb_laws.sum()

2215

In [344]:
v02 = pd.read_csv('data/v0.2/law.csv')
v02 = v02[v02.sysid!='None'].reset_index(drop=True)
v02['sysid'] = v02.sysid.map(lambda x: int(x)) 
v02

,sysid,title,txt
0,726890,กฎบัตรสหประชาชาติ ซึ่งรับตกลงกัน ณ เมืองซานฟาน...,\n\n\n\n\n\n\n\n\nกฎบัตรสหประชาชาติ\n\n\n\n\n\...
1,301085,กฎบัตรสหประชาชาติ ค.ศ. 1945,\n\n\n\n\n\n\n\n\n \n\n\n\n\n\n\n \n \n \n \n ...
2,740727,ประกาศกระทรวงมหาดไทย เรื่อง การออกหนังสือผ่านแ...,\n\n\n\n\n\n\n\n\nประกาศกระทรวงมหาดไทย\n\n\n\n...
3,497860,ประกาศการมีผลใช้บังคับของความตกลงระหว่างรัฐบาล...,\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\nประกาศ\r\n...
4,490191,ประกาศ การมีผลใช้บังคับของความตกลง ระหว่าง รัฐ...,\n\n\n\n\n\n\n\n\nประกาศ\n\n\n\n\n\n\nประกาศ\n...
...,...,...,...
42749,771111,ข้อบัญญัติองค์การบริหารส่วนตำบลนิลเพชร เรื่อง ...,\n\n\n\n\n\n\n\n\nข้อบัญญัติองค์การบริหารส่วนต...
42750,771127,ข้อบัญญัติองค์การบริหารส่วนตำบลนิลเพชร เรื่อง ...,\n\n\n\n\n\n\n\n\nข้อบัญญัติองค์การบริหารส่วนต...
42751,771129,ข้อบัญญัติองค์การบริหารส่วนตำบลนิลเพชร เรื่อง ...,\n\n\n\n\n\n\n\n\nข้อบัญญัติองค์การบริหารส่วนต...
42752,771099,ข้อบัญญัติองค์การบริหารส่วนตำบลสว่างแดนดิน เรื...,\n\n\n\n\n\n\n\n\nข้อบัญญัติองค์การบริหารส่วนต...


In [345]:
law_url_df[law_url_df.law_type=='ประมวลกฎหมาย'].drop('law_group',1).drop_duplicates().merge(v02, how='left',on='sysid')

,title_x,law_url,sysid,law_type,title_y,txt
0,ประมวลกฎหมายอาญา (ฉบับ Update ล่าสุด),https://www.krisdika.go.th/librarian/get?sysid...,443287,ประมวลกฎหมาย,NaN,NaN
1,ประมวลกฎหมายวิธีพิจารณาความอาญา (ฉบับ Update ล...,https://www.krisdika.go.th/librarian/get?sysid...,570066,ประมวลกฎหมาย,NaN,NaN
2,ประมวลกฎหมายอาญาทหาร (ฉบับ Update ล่าสุด),https://www.krisdika.go.th/librarian/get?sysid...,320572,ประมวลกฎหมาย,NaN,NaN
3,ประมวลกฎหมายที่ดิน (ฉบับ Update ล่าสุด),https://www.krisdika.go.th/librarian/get?sysid...,570844,ประมวลกฎหมาย,NaN,NaN
4,ประมวลกฎหมายแพ่งและพาณิชย์ (ฉบับ Update ล่าสุด),https://www.krisdika.go.th/librarian/get?sysid...,567410,ประมวลกฎหมาย,NaN,NaN
5,ประมวลกฎหมายวิธีพิจารณาความแพ่ง (ฉบับ Update ล...,https://www.krisdika.go.th/librarian/get?sysid...,570093,ประมวลกฎหมาย,NaN,NaN


In [347]:
#designate law types
law_types = ['รัฐธรรมนูญ', 'พระราชบัญญัติ', 'พระราชกำหนด','ประมวลกฎหมาย', 'ประมวลรัษฎากร','ประกาศ','คำสั่ง','พระธรรมนูญ']

def get_law_type(law_name, law_types=law_types, max_char=13):
    for l in law_types:
        if l in law_name[:max_char]:
            return l
    return 'others'

v02['law_type'] = v02.title.map(get_law_type)

In [348]:
v02.law_type.value_counts()

ประกาศ           22729
others           16705
พระราชบัญญัติ     2028
คำสั่ง            1170
พระราชกำหนด        103
พระธรรมนูญ          19
Name: law_type, dtype: int64

In [352]:
v02[v02.law_type=='others'].title.map(lambda x: x[:10]).value_counts().head(20)

พระราชกฤษฎ    3916
กฎกระทรวง     2438
ข้อบัญญัติ    1958
กฎกระทรวงก    1158
ระเบียบคณะ     980
เทศบัญญัติ     752
ระเบียบกรม     637
ระเบียบกระ     551
กฎกระทรวงใ     360
ข้อบังคับเ     279
ระเบียบสำน     266
ระเบียบการ     227
ระเบียบองค     206
ระเบียบเทศ     204
ระเบียบกรุ     198
ข้อกำหนดเก     196
ข้อบังคับม     148
ข้อบังคับก     147
กฎกระทรวงม     125
ข้อบังคับส     125
Name: title, dtype: int64

In [358]:
pd.set_option('display.max_colwidth', -1)
v02[v02.title.map(lambda x: 'พระราชบัญญัติเครื่องหมายการค้า พ.ศ. 2534' in x)][['sysid','title','law_type']]

/home/cstorm125/venv/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


,sysid,title,law_type
24706,750158,พระราชบัญญัติเครื่องหมายการค้า พ.ศ. 2534 (ฉบับ Update ณ วันที่ 01/04/2543)\r\n,พระราชบัญญัติ
24708,302660,พระราชบัญญัติเครื่องหมายการค้า พ.ศ. 2534,พระราชบัญญัติ
24709,312094,กฎกระทรวง ฉบับที่ 5 (พ.ศ. 2543) ออกตามความในพระราชบัญญัติเครื่องหมายการค้า พ.ศ. 2534,others
24710,302664,กฎกระทรวง ฉบับที่ 3 (พ.ศ.2540) ออกตามความในพระราชบัญญัติเครื่องหมายการค้า พ.ศ. 2534,others
24712,312093,กฎกระทรวง ฉบับที่ 4 (พ.ศ. 2543) ออกตามความในพระราชบัญญัติเครื่องหมายการค้า พ.ศ. 2534,others
24714,302662,กฎกระทรวง (พ.ศ. 2535) ออกตามความในพระราชบัญญัติเครื่องหมายการค้า พ.ศ. 2534,others
24715,784543,กฎกระทรวง ฉบับที่ 5 (พ.ศ. 2560) ออกตามความในพระราชบัญญัติเครื่องหมายการค้า พ.ศ. 2534,others
24718,302663,กฎกระทรวง ฉบับที่ 2 (พ.ศ. 2535) ออกตามความในพระราชบัญญัติเครื่องหมายการค้า พ.ศ. 2534,others
24719,606102,กฎกระทรวง ฉบับที่ 3 (พ.ศ. 2540) ออกตามความในพระราชบัญญัติเครื่องหมายการค้า พ.ศ. 2534 (ฉบับ Update ล่าสุด),others
24725,606100,กฎกระทรวง (พ.ศ. 2535) ออกตามความในพระราชบัญญัติเครื่องหมายการค้า พ.ศ. 2534 (ฉบับ Update ล่าสุด),others


In [369]:
print(v02[v02.sysid==750158].txt.item())










พระราชบัญญัติ






พระราชบัญญัติ
เครื่องหมายการค้า
พ.ศ. ๒๕๓๔
                   
 
ภูมิพลอดุลยเดช ป.ร.
ให้ไว้ ณ วันที่ ๒๘ ตุลาคม
พ.ศ. ๒๕๓๔
เป็นปีที่ ๔๖
ในรัชกาลปัจจุบัน
 
พระบาทสมเด็จพระปรมินทรมหาภูมิพลอดุลยเดช
มีพระบรมราชโองการโปรดเกล้า ฯ ให้ประกาศว่า
 
โดยที่เป็นการสมควรปรับปรุงกฎหมายว่าด้วยเครื่องหมายการค้า
 
จึงทรงพระกรุณาโปรดเกล้าฯ
ให้ตราพระราชบัญญัติขึ้นไว้โดยคำแนะนำและยินยอมของสภานิติบัญญัติแห่งชาติ
ดังต่อไปนี้
 
มาตรา ๑ 
พระราชบัญญัตินี้เรียกว่า พระราชบัญญัติเครื่องหมายการค้า
พ.ศ.๒๕๓๔
 
มาตรา ๒[๑] 
พระราชบัญญัตินี้ให้ใช้บังคับเมื่อพ้นกำหนดเก้าสิบวันนับแต่วันประกาศในราชกิจจานุเบกษาเป็นต้นไป
 
มาตรา ๓ 
ให้ยกเลิก
(๑) พระราชบัญญัติเครื่องหมายการค้า พุทธศักราช ๒๔๗๔
(๒) พระราชบัญญัติเครื่องหมายการค้า (ฉบับที่ ๓) พ.ศ. ๒๕๐๔
บรรดาบทกฎหมาย
กฎ และข้อบังคับอื่นในส่วนที่มีบัญญัติไว้แล้วในพระราชบัญญัตินี้ หรือซึ่งขัดหรือแย้งกับบทแห่งพระราชบัญญัตินี้
ให้ใช้พระราชบัญญัตินี้แทน
 
มาตรา ๔ 
ในพระราชบัญญัตินี้
เครื่องหมาย[๒] หมายความว่า ภาพถ่าย ภาพวาด ภาพประดิษฐ์ ตรา ชื่อ คำ ข้อความ 

## Cross-check with [DIP](https://www.ipthailand.go.th/)

In [573]:
headers = {"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36"}
url = 'https://www.ipthailand.go.th'

In [470]:
with requests.get(f'{url}/th/dip-law-2.html',verify=False,headers=headers) as r:
    soup = BeautifulSoup(r.content, 'html.parser')

lev1_df = pd.DataFrame([(i.text.strip(), f"{url}{i.find('a').get('href')}") \
                          for i in soup.find_all('h2', class_='uk-h3 uk-margin-remove')])
lev1_df.columns = ['lev1','lev1_url']
lev1_df

,lev1,lev1_url
0,กฎหมาย - เครื่องหมายการค้า,https://www.ipthailand.go.th/th/trademark-008.html
1,กฎหมาย - สิทธิบัตร,https://www.ipthailand.go.th/th/patent-008.html
2,กฎหมาย - สิทธิบัตรการออกแบบผลิตภัณฑ์,https://www.ipthailand.go.th/th/design-patent-010.html
3,กฎหมาย - ลิขสิทธิ์,https://www.ipthailand.go.th/th/copyright-007.html
4,กฎหมาย - สิ่งบ่งชี้ทางภูมิศาสตร์,https://www.ipthailand.go.th/th/gi-010.html
5,กฎหมาย - การป้องปรามการละเมิดทรัพย์สินทางปัญญา,https://www.ipthailand.go.th/th/dip-law-2/category/กฎหมาย-ประกาศ-ระเบียบ-คำสั่ง-การป้องปรามการละเมิดทรัพย์สินทางปัญญา.html
6,กฎหมาย - ความลับทางการค้า,https://www.ipthailand.go.th/th/secret-002.html
7,กฎหมาย - แบบผังภูมิของวงจรรวม,https://www.ipthailand.go.th/th/layout-007.html
8,กฎหมาย - การผลิตผลิตภัณฑ์ซีดี,https://www.ipthailand.go.th/th/cd-008.html
9,ประกาศกรมทรัพย์สินทางปัญญา,https://www.ipthailand.go.th/th/ประกาศกรมทรัพย์สินทางปัญญา.html


In [566]:
lev2_dfs = []
for row in tqdm(lev1_df.itertuples(index=False)):
    with requests.get(row[1],verify=False,headers=headers) as r:
        soup = BeautifulSoup(r.content, 'html.parser')
    
    #if it's level 2
    d = pd.DataFrame([(i.text.strip(), f"{url}{i.find('a').get('href')}") \
                                  for i in soup.find_all('h2', class_='uk-h3 uk-margin-remove')])
    #if it's actually level 3; manually checked there is only one page each, no pagination needed
    if len(d)==0:
        d = pd.DataFrame([(i.text.strip(), f"{url}{i.find('a').get('href')}") \
                                  for i in soup.find_all('h3', class_='uk-margin-remove')])
        d.columns = ['lev3','lev3_url']
        d['lev1'] = row[0]
        d['lev1_url'] = row[1]
        d['lev2'] = None
        d['lev2_url'] = None
    else:
        d.columns = ['lev2','lev2_url']
        d['lev1'] = row[0]
        d['lev1_url'] = row[1]
        d['lev3'] = None
        d['lev3_url'] = None
    
    #append to list
    d = d[['lev1','lev2','lev3','lev1_url','lev2_url','lev3_url']]
    lev2_dfs.append(d)
    
lev2_df = pd.concat(lev2_dfs)
lev2_df

In [567]:
#those that are actually level 3
lev2_df[~lev2_df.lev3_url.isna()].tail()

,lev1,lev2,lev3,lev1_url,lev2_url,lev3_url
67,คำสั่งกรมทรัพย์สินทางปัญญา,NaN,คำสั่งกรมทรัพย์สินทางปัญญา เรื่อง แต่งตั้งคณะกรรมการกลั่นกรองการกำหนดคุณลักษณะของพัสดุที่จะจัดซื้อและงานที่จะจัดจ้าง,https://www.ipthailand.go.th/th/คำสั่งกรมทรัพย์สินทางปัญญา.html,NaN,https://www.ipthailand.go.th/th/คำสั่งกรมทรัพย์สินทางปัญญา/item/461-2563.html
68,คำสั่งกรมทรัพย์สินทางปัญญา,NaN,จัดตั้งกลุ่มงานคุ้มครองจริยธรรมประจำกรมทรัพย์สินทางปัญญา,https://www.ipthailand.go.th/th/คำสั่งกรมทรัพย์สินทางปัญญา.html,NaN,https://www.ipthailand.go.th/th/คำสั่งกรมทรัพย์สินทางปัญญา/item/จัดตั้งกลุ่มงานคุ้มครองจริยธรรมประจำกรมทรัพย์สินทางปัญญา.html
69,คำสั่งกรมทรัพย์สินทางปัญญา,NaN,มอบหมายเจ้าหน้าทีกรมทรัพย์สินทางปัญญาให้ปฏิบัติหน้าที่เป็นเจ้าหน้าที่ออกใบรับรองอิเล็กทรอนิกส์ และเจ้าหน้าที่รับลงทะเบียน,https://www.ipthailand.go.th/th/คำสั่งกรมทรัพย์สินทางปัญญา.html,NaN,https://www.ipthailand.go.th/th/คำสั่งกรมทรัพย์สินทางปัญญา/item/มอบหมายเจ้าหน้าทีกรมทรัพย์สินทางปัญญาให้ปฏิบัติหน้าที่เป็นเจ้าหน้าที่ออกใบรับรองอิเล็กทรอนิกส์-และเจ้าหน้าที่รับลงทะเบียน.html
70,คำสั่งกรมทรัพย์สินทางปัญญา,NaN,คำสั่งกรมทรัพย์สินทางปัญญา ที่ ๑๔๐/๒๕๖๐,https://www.ipthailand.go.th/th/คำสั่งกรมทรัพย์สินทางปัญญา.html,NaN,https://www.ipthailand.go.th/th/คำสั่งกรมทรัพย์สินทางปัญญา/item/คำสั่งกรมทรัพย์สินทางปัญญา-ที่-๑๔๐-๒๕๖๐.html
78,ประกาศรายชื่อกฎหมายและหน่วยงานที่รับผิดชอบการประเมินผลสัมฤทธิ์ของกฎหมายฯ,NaN,ประกาศรายชื่อกฎหมายและหน่วยงานที่รับผิดชอบการประเมินผลสัมฤทธิ์ของกฎหมายฯ,https://www.ipthailand.go.th/th/dip-law-2/category/list-legal.html,NaN,https://www.ipthailand.go.th/th/list-legal.html


In [569]:
lev2_df[lev2_df.lev3_url.isna()].groupby('lev1').lev2.count().reset_index()

,lev1,lev2
0,กฎหมาย - การผลิตผลิตภัณฑ์ซีดี,6
1,กฎหมาย - ความลับทางการค้า,2
2,กฎหมาย - ลิขสิทธิ์,9
3,กฎหมาย - สิทธิบัตร,9
4,กฎหมาย - สิทธิบัตรการออกแบบผลิตภัณฑ์,8
5,กฎหมาย - สิ่งบ่งชี้ทางภูมิศาสตร์,6
6,กฎหมาย - เครื่องหมายการค้า,9
7,กฎหมาย - แบบผังภูมิของวงจรรวม,4
8,ร่างกฎหมายที่รับฟังความคิดเห็น,7


In [570]:
lev2_df[lev2_df.lev3.isna()].head()

,lev1,lev2,lev3,lev1_url,lev2_url,lev3_url
0,กฎหมาย - เครื่องหมายการค้า,พระราชบัญญัติ (Acts),NaN,https://www.ipthailand.go.th/th/trademark-008.html,https://www.ipthailand.go.th/th/dip-law-2/category/พระราชบัญญัติ-acts-2.html,NaN
1,กฎหมาย - เครื่องหมายการค้า,กฎกระทรวง (Ministerial regulations),NaN,https://www.ipthailand.go.th/th/trademark-008.html,https://www.ipthailand.go.th/th/dip-law-2/category/กฎกระทรวง-ministerial-regulations-2.html,NaN
2,กฎหมาย - เครื่องหมายการค้า,ประกาศกระทรวงพาณิชย์ (Notifications of MOC),NaN,https://www.ipthailand.go.th/th/trademark-008.html,https://www.ipthailand.go.th/th/dip-law-2/category/ประกาศกระทรวงพาณิชย์-notifications-of-moc-2.html,NaN
3,กฎหมาย - เครื่องหมายการค้า,ประกาศกรมทรัพย์สินทางปัญญา (Notifications of DIP),NaN,https://www.ipthailand.go.th/th/trademark-008.html,https://www.ipthailand.go.th/th/dip-law-2/category/ประกาศกรมทรัพย์สินทางปัญญา-notifications-of-dip-2.html,NaN
4,กฎหมาย - เครื่องหมายการค้า,ระเบียบกรมทรัพย์สินทางปัญญา (Rules of DIP),NaN,https://www.ipthailand.go.th/th/trademark-008.html,https://www.ipthailand.go.th/th/dip-law-2/category/ระเบียบกรมทรัพย์สินทางปัญญา-rules-of-dip-2.html,NaN


In [515]:
#check how many pages
pages = []
for row in tqdm(lev2_df[lev2_df.lev3.isna()].itertuples(index=False)):
    with requests.get(row[4],verify=False,headers=headers) as r:
        soup = BeautifulSoup(r.content, 'html.parser')
    ul = soup.find('ul', class_='uk-pagination')
    if ul is None:
        nb_page = 1
    else:
        nb_page = int([i.text for i in ul.find_all('li')[0]][-3])
    p = {'lev1':row[0],'lev2':row[1],'nb_pages':nb_page}
    pages.append(p)

In [571]:
page_df = pd.DataFrame(pages)
lev2_toscrape = lev2_df.merge(page_df, on =['lev1','lev2'])
#equal to things to scrape
lev2_toscrape.shape, lev2_df[lev2_df.lev3.isna()].shape

((60, 7), (60, 6))

In [572]:
lev2_toscrape.tail()

,lev1,lev2,lev3,lev1_url,lev2_url,lev3_url,nb_pages
55,ร่างกฎหมายที่รับฟังความคิดเห็น,เอกสารประกอบการเสนอร่างกฎหมายทรัพย์สินทางปัญญาตามแนวทางการจัดทำและการเสนอร่างกฎหมายตามมาตรา ๗๗ ของรัฐธรรมนูญแห่งราชอาณาจักรไทย,NaN,https://www.ipthailand.go.th/th/ร่างกฎหมายที่รับฟังความคิดเห็น.html,https://www.ipthailand.go.th/th/dip-law-2/category/เอกสารประกอบการเสนอร่างกฎหมายทรัพย์สินทางปัญญาตามแนวทางการจัดทำและการเสนอร่างกฎหมายตามมาตรา-๗๗-ของรัฐธรรมนูญแห่งราชอาณาจักรไทย.html,NaN,1
56,ร่างกฎหมายที่รับฟังความคิดเห็น,สรุปผลการรับฟังความคิดเห็นร่างพระราชบัญญัติลิขสิทธิ์ (ฉบับที่..) พ.ศ. .... (ในส่วนที่เกี่ยวกับข้อยกเว้นความรับผิดของผู้ให้บริการและการเข้าเป็นภาคีสนธิสัญญาว่าด้วยลิขสิทธิ์ขององค์การทรัพย์สินทางปัญญาโลก) ที่ผ่านการพิจารณาของคณะกรรมการกฤษฎีกา (คณะที่ ๖),NaN,https://www.ipthailand.go.th/th/ร่างกฎหมายที่รับฟังความคิดเห็น.html,https://www.ipthailand.go.th/th/dip-law-2/category/สรุปผลการรับฟังความคิดเห็นร่างพระราชบัญญัติลิขสิทธิ์-ฉบับที่-พ-ศ-ในส่วนที่เกี่ยวกับข้อยกเว้นความรับผิดของผู้ให้บริการและการเข้าเป็นภาคีสนธิสัญญาว่าด้วยลิขสิทธิ์ขององค์การทรัพย์สินทางปัญญาโลก-ที่ผ่านการพิจารณาของคณะกรรมการกฤษฎีกา-คณะที่-๖.html,NaN,1
57,ร่างกฎหมายที่รับฟังความคิดเห็น,รับฟังความคิดเห็นต่อร่างกฎหมายทรัพย์สินทางปัญญา,NaN,https://www.ipthailand.go.th/th/ร่างกฎหมายที่รับฟังความคิดเห็น.html,https://www.ipthailand.go.th/th/dip-law-2/category/รับฟังความคิดเห็นต่อร่างกฎหมายทรัพย์สินทางปัญญา-2.html,NaN,1
58,ร่างกฎหมายที่รับฟังความคิดเห็น,เอกสารประกอบการเสนอร่างกฎหมายทรัพย์สินทางปัญญาตามแนวทางการจัดทำและการเสนอร่างกฎหมายตามมาตรา 77 ของรัฐธรรมนูญแห่งราชอาณาจักรไทย,NaN,https://www.ipthailand.go.th/th/ร่างกฎหมายที่รับฟังความคิดเห็น.html,https://www.ipthailand.go.th/th/dip-law-2/category/เอกสารประกอบการเสนอร่างกฎหมายทรัพย์สินทางปัญญาตามแนวทางการจัดทำและการเสนอร่างกฎหมายตามมาตรา-77-ของรัฐธรรมนูญแห่งราชอาณาจักรไทย.html,NaN,1
59,ร่างกฎหมายที่รับฟังความคิดเห็น,เอกสารประกอบการเสนอร่างพระราชบัญญัติสิทธิบัตร (ฉบับที่..) พ.ศ. .... ตามแนวทางการจัดทำและการเสนอร่างกฎหมายตามมาตรา 77 ของรัฐธรรมนูญแห่งราชอาณาจักรไทย,NaN,https://www.ipthailand.go.th/th/ร่างกฎหมายที่รับฟังความคิดเห็น.html,https://www.ipthailand.go.th/th/dip-law-2/category/เอกสารประกอบการเสนอร่างพระราชบัญญัติสิทธิบัตร-ฉบับที่-พ-ศ-ตามแนวทางการจัดทำและการเสนอร่างกฎหมายตามมาตรา-77-ของรัฐธรรมนูญแห่งราชอาณาจักรไทย.html,NaN,1


In [576]:
lev3_dfs = []

for row in tqdm(lev2_toscrape.itertuples(index=False)):
    for i in range(1,row[6]+1):
        sub_url = f'{row[4][:-5]}/{i}.html'
        with requests.get(sub_url,verify=False,headers=headers) as r:
            soup = BeautifulSoup(r.content, 'html.parser')
        try:
            d= pd.DataFrame([(i.text.strip(), f"{url}{i.find('a').get('href')}") \
                                      for i in soup.find_all('h3', class_='uk-margin-remove')])
            d.columns = ['lev3','lev3_url']
            d['lev1'] = row[0]
            d['lev2'] = row[1]
            d['lev1_url'] = row[3]
            d['lev2_url'] = row[2]
            d = d[['lev1','lev2','lev3','lev1_url','lev2_url','lev3_url']]
            lev3_dfs.append(d)
        except:
            continue
lev3_df = pd.concat(lev3_dfs)
lev3_df = lev3_df[['lev1','lev2','lev3','lev3_url']].reset_index(drop=True)
lev3_df

,lev1,lev2,lev3,lev3_url
0,กฎหมาย - เครื่องหมายการค้า,พระราชบัญญัติ (Acts),พระราชบัญญัติเครื่องหมายการค้า พ.ศ. 2534 แก้ไขเพิ่มเติมโดยพระราชบัญญัติเครื่องหมายการค้า (ฉบับที่ 2) พ.ศ. 2543 และแก้ไขเพิ่มเติมโดยพระราชบัญญัติเครื่องหมายการค้า (ฉบับที่ 3) พ.ศ. 2559,https://www.ipthailand.go.th/th/dip-law-2/item/พระราชบัญญัติเครื่องหมายการค้า-พ-ศ-๒๕๕๙-ฉบับที่3.html
1,กฎหมาย - เครื่องหมายการค้า,พระราชบัญญัติ (Acts),TRADEMARK ACT B.E. 2534 AMENDED BY TRADEMARK ACT (NO. 2) B.E. 2543 AND TRADEMARK ACT (NO. 3) B.E. 2559,https://www.ipthailand.go.th/th/dip-law-2/item/trademark-act-b-e-2534-amended-by-trademark-act-no-2-b-e-2543-and-trademark-act-no-3-b-e-2559.html
2,กฎหมาย - เครื่องหมายการค้า,พระราชบัญญัติ (Acts),พระราชบัญญัติเครื่องหมายการค้า พ.ศ. 2534,https://www.ipthailand.go.th/th/dip-law-2/item/พระราชบัญญัติ-เครื่องหมายการค้า-พ-ศ-2543.html
3,กฎหมาย - เครื่องหมายการค้า,พระราชบัญญัติ (Acts),Trademark Act B.E. 2534 (1991),https://www.ipthailand.go.th/th/dip-law-2/item/trademark-act-b-e-2534-1991.html
4,กฎหมาย - เครื่องหมายการค้า,พระราชบัญญัติ (Acts),พระราชบัญญัติเครื่องหมายการค้า พ.ศ. 2543,https://www.ipthailand.go.th/th/dip-law-2/item/พระราชบัญญัติเครื่องหมายการค้า-พ-ศ-๒๕๓๔-แก้ไขเพิ่มเติมโดยพระราชบัญญัติ-เครื่องหมายการค้า-ฉบับที่2.html
...,...,...,...,...
359,ร่างกฎหมายที่รับฟังความคิดเห็น,สรุปผลการรับฟังความคิดเห็นร่างพระราชบัญญัติลิขสิทธิ์ (ฉบับที่..) พ.ศ. .... (ในส่วนที่เกี่ยวกับข้อยกเว้นความรับผิดของผู้ให้บริการและการเข้าเป็นภาคีสนธิสัญญาว่าด้วยลิขสิทธิ์ขององค์การทรัพย์สินทางปัญญาโลก) ที่ผ่านการพิจารณาของคณะกรรมการกฤษฎีกา (คณะที่ ๖),ร่างพระราชบัญญัติลิขสิทธิ์ (ฉบับที่..) พ.ศ. .... (ในส่วนที่เกี่ยวกับข้อยกเว้นความรับผิดของผู้ให้บริการและการเข้าเป็นภาคีสนธิสัญญาว่าด้วยลิขสิทธิ์ขององค์การทรัพย์สินทางปัญญาโลก) ที่ผ่านการพิจารณาของคณะกรรมการกฤษฎีกา (คณะที่ ๖),https://www.ipthailand.go.th/th/dip-law-2/item/ร่างพระราชบัญญัติลิขสิทธิ์-ฉบับที่-พ-ศ-ในส่วนที่เกี่ยวกับข้อยกเว้นความรับผิดของผู้ให้บริการและการเข้าเป็นภาคีสนธิสัญญาว่าด้วยลิขสิทธิ์ขององค์การทรัพย์สินทางปัญญาโลก-ที่ผ่านการพิจารณาของคณะกรรมการกฤษฎีกา-คณะที่-๖.html
360,ร่างกฎหมายที่รับฟังความคิดเห็น,เอกสารประกอบการเสนอร่างกฎหมายทรัพย์สินทางปัญญาตามแนวทางการจัดทำและการเสนอร่างกฎหมายตามมาตรา 77 ของรัฐธรรมนูญแห่งราชอาณาจักรไทย,เผยแพร่สรุปผลการรับฟังความคิดเห็นการเข้าเป็นภาคีสนธิสัญญาว่าด้วยลิขสิทธิ์ขององค์การทรัพย์สินทางปัญญาโลก (WIPO Copyright Treaty),https://www.ipthailand.go.th/th/dip-law-2/item/เผยแพร่สรุปผลการรับฟังความคิดเห็นการเข้าเป็นภาคีสนธิสัญญาว่าด้วยลิขสิทธิ์ขององค์การทรัพย์สินทางปัญญาโลก-wipo-copyright-treaty.html?category_id=2688
361,ร่างกฎหมายที่รับฟังความคิดเห็น,เอกสารประกอบการเสนอร่างกฎหมายทรัพย์สินทางปัญญาตามแนวทางการจัดทำและการเสนอร่างกฎหมายตามมาตรา 77 ของรัฐธรรมนูญแห่งราชอาณาจักรไทย,เผยแพร่สรุปผลการรับฟังความคิดเห็น ร่างพระราชบัญญัติลิขสิทธิ์(ฉบับที่.. ) พ.ศ. .... (ในส่วนที่เกี่ยวกับข้อจำกัดความรับผิดของ ผู้ให้บริการ และการเข้าเป็นภาคีสนธิสัญญาว่าด้วยลิขสิทธิ์ขององค์การทรัพย์สินทางปัญญาโลก),https://www.ipthailand.go.th/th/dip-law-2/item/เผยแพร่สรุปผลการรับฟังความคิดเห็น-ร่างพระราชบัญญัติลิขสิทธิ์-ฉบับที่-พ-ศ-ในส่วนที่เกี่ยวกับข้อจำกัดความรับผิดของ-ผู้ให้บริการ-และการเข้าเป็นภาคีสนธิสัญญาว่าด้วยลิขสิทธิ์ขององค์การทรัพย์สินทางปัญญาโลก.html?category_id=2688
362,ร่างกฎหมายที่รับฟังความคิดเห็น,เอกสารประกอบการเสนอร่างกฎหมายทรัพย์สินทางปัญญาตามแนวทางการจัดทำและการเสนอร่างกฎหมายตามมาตรา 77 ของรัฐธรรมนูญแห่งราชอาณาจักรไทย,ร่างพระราชบัญญัติลิขสิทธิ์(ฉบับที่.. ) พ.ศ. .... (ในส่วนที่เกี่ยวกับข้อยกเว้นการละเมืดลิขสิทธิ์สำหรับคนพิการ),https://www.ipthailand.go.th/th/dip-law-2/item/ร่างพระราชบัญญัติลิขสิทธิ์-ฉบับที่-พ-ศ-ในส่วนที่เกี่ยวกับข้อยกเว้นการละเมืดลิขสิทธิ์สำหรับคนพิการ.html?category_id=2688


In [577]:
lev2_df2 = lev2_df[~lev2_df.lev3.isna()]
lev3_df2 = lev3_df.merge(lev2_df.drop(['lev3','lev3_url'],1),on=['lev1','lev2'])
lev3_df2 = lev3_df2[['lev1','lev2','lev3','lev1_url','lev2_url','lev3_url']]
ip_df = pd.concat([lev2_df2,lev3_df2]).sort_values(['lev1','lev2','lev3']).reset_index(drop=True)
ip_df.shape

(383, 6)

In [579]:
ip_df

,lev1,lev2,lev3,lev1_url,lev2_url,lev3_url
0,กฎหมาย - การป้องปรามการละเมิดทรัพย์สินทางปัญญา,NaN,กฎหมายที่เกี่ยวกับการป้องปรามการละเมิดทรัพย์สินทางปัญญา,https://www.ipthailand.go.th/th/dip-law-2/category/กฎหมาย-ประกาศ-ระเบียบ-คำสั่ง-การป้องปรามการละเมิดทรัพย์สินทางปัญญา.html,NaN,https://www.ipthailand.go.th/th/protect-002.html
1,กฎหมาย - การผลิตผลิตภัณฑ์ซีดี,กฏกระทรวง (Ministerial regulations),กฎกระทรวงว่าด้วยการกำหนดผลิตภัณฑ์ซีดีและอุปกรณ์ของเครื่องจักร พ.ศ. 2548 (ลงวันที่ 24 สิงหาคม 2548),https://www.ipthailand.go.th/th/cd-008.html,https://www.ipthailand.go.th/th/dip-law-2/category/กฏกระทรวง-ministerial-regulations.html,https://www.ipthailand.go.th/th/dip-law-2/item/กฎกระทรวง-ว่าด้วยการกำหนดผลิตภัณฑ์ซีดีและอุปกรณ์ของเครื่องจักร-พ-ศ-๒๕๔๘.html
2,กฎหมาย - การผลิตผลิตภัณฑ์ซีดี,คำสั่งกรมทรัพย์สินทางปัญญา (Orders of DIP),คำสั่งกรมทรัพย์สินทางปัญญา ที่ 152/2548 เรื่อง มอบอำนาจให้ผู้ว่าราชการจังหวัดปฏิบัติราชการแทน (เพิ่มเติม) (ลงวันที่ 23 สิงหาคม 2548),https://www.ipthailand.go.th/th/cd-008.html,https://www.ipthailand.go.th/th/dip-law-2/category/คำสั่งกรมทรัพย์สินทางปัญญา-orders-of-dip-3.html,https://www.ipthailand.go.th/th/dip-law-2/item/คำสั่งกรม-ที่-152-2548-เรื่อง-มอบอำนาจให้ผู้ว่าราชการจังหวัดปฏิบัติราชการแทน-เพิ่มเติม.html
3,กฎหมาย - การผลิตผลิตภัณฑ์ซีดี,คำสั่งกรมทรัพย์สินทางปัญญา (Orders of DIP),คำสั่งกรมทรัพย์สินทางปัญญา ที่ 189/2553 เรื่อง มอบอำนาจให้ข้าราชการปฏิบัติราชการตามพระราชบัญญัติการผลิตผลิตภัณฑ์ซีดี พ.ศ. 2548 แทนอธิบดีกรมทรัพย์สินทางปัญญา (ลงวันที่ 28 เมษายน 2553),https://www.ipthailand.go.th/th/cd-008.html,https://www.ipthailand.go.th/th/dip-law-2/category/คำสั่งกรมทรัพย์สินทางปัญญา-orders-of-dip-3.html,https://www.ipthailand.go.th/th/dip-law-2/item/คำสั่งกรมทรัพย์สินทางปัญญาที่-๑๘๙-๒๕๕๓-เรื่อง-มอบอำนาจให้ข้าราชการปฏิบัติราชการตามพระราชบัญญัติการผล.html
4,กฎหมาย - การผลิตผลิตภัณฑ์ซีดี,คำสั่งกรมทรัพย์สินทางปัญญา (Orders of DIP),คำสั่งกรมทรัพย์สินทางปัญญา ที่ 229/2553 เรื่อง ให้ข้าราชการในสังกัดสำนักงานพาณิชย์จังหวัดปฏิบัติหน้าที่ราชการในฐานะพนักงานเจ้าหน้าที่และมีบัตรประจำตัวพนักงานเจ้าหน้าที่ตามพระราชบัญญัติการผลิตผลิตภัณฑ์ซีดี พ.ศ. 2548 (ลงวันที่ 14 พฤษภาคม 2553),https://www.ipthailand.go.th/th/cd-008.html,https://www.ipthailand.go.th/th/dip-law-2/category/คำสั่งกรมทรัพย์สินทางปัญญา-orders-of-dip-3.html,https://www.ipthailand.go.th/th/dip-law-2/item/คำสั่งกรมทรัพย์สินทางปัญญา-ที่-๒๒๙-๒๕๕๓-เรื่อง-ให้ข้าราชการในสังกัดสำนักงานพาณิชย์จังหวัดปฏิบัติหน้า.html
...,...,...,...,...,...,...
378,ร่างกฎหมายที่รับฟังความคิดเห็น,เอกสารประกอบการเสนอร่างกฎหมายทรัพย์สินทางปัญญาตามแนวทางการจัดทำและการเสนอร่างกฎหมายตามมาตรา 77 ของรัฐธรรมนูญแห่งราชอาณาจักรไทย,ร่างพระราชบัญญัติลิขสิทธิ์(ฉบับที่.. ) พ.ศ. .... (ในส่วนที่เกี่ยวกับข้อยกเว้นการละเมืดลิขสิทธิ์สำหรับคนพิการ),https://www.ipthailand.go.th/th/ร่างกฎหมายที่รับฟังความคิดเห็น.html,https://www.ipthailand.go.th/th/dip-law-2/category/เอกสารประกอบการเสนอร่างกฎหมายทรัพย์สินทางปัญญาตามแนวทางการจัดทำและการเสนอร่างกฎหมายตามมาตรา-77-ของรัฐธรรมนูญแห่งราชอาณาจักรไทย.html,https://www.ipthailand.go.th/th/dip-law-2/item/ร่างพระราชบัญญัติลิขสิทธิ์-ฉบับที่-พ-ศ-ในส่วนที่เกี่ยวกับข้อยกเว้นการละเมืดลิขสิทธิ์สำหรับคนพิการ.html?category_id=2688
379,ร่างกฎหมายที่รับฟังความคิดเห็น,เอกสารประกอบการเสนอร่างกฎหมายทรัพย์สินทางปัญญาตามแนวทางการจัดทำและการเสนอร่างกฎหมายตามมาตรา 77 ของรัฐธรรมนูญแห่งราชอาณาจักรไทย,เผยแพร่สรุปผลการรับฟังความคิดเห็น ร่างพระราชบัญญัติลิขสิทธิ์(ฉบับที่.. ) พ.ศ. .... (ในส่วนที่เกี่ยวกับข้อจำกัดความรับผิดของ ผู้ให้บริการ และการเข้าเป็นภาคีสนธิสัญญาว่าด้วยลิขสิทธิ์ขององค์การทรัพย์สินทางปัญญาโลก),https://www.ipthailand.go.th/th/ร่างกฎหมายที่รับฟังความคิดเห็น.html,https://www.ipthailand.go.th/th/dip-law-2/category/เอกสารประกอบการเสนอร่างกฎหมายทรัพย์สินทางปัญญาตามแนวทางการจัดทำและการเสนอร่างกฎหมายตามมาตรา-77-ของรัฐธรรมนูญแห่งราชอาณาจักรไทย.html,https://www.ipthailand.go.th/th/dip-law-2/item/เผยแพร่สรุปผลการรับฟังความคิดเห็น-ร่างพระราชบัญญัติลิขสิทธิ์-ฉบับที่-พ-ศ-ในส่วนที่เกี่ยวกับข้อจำกัดความรับผิดของ-ผู้ให้บริการ-และการเข้าเป็นภาคีสนธิสัญญาว่าด้วยลิขสิทธิ์ขององค์การทรัพย์สินทางปัญญาโลก.

In [578]:
# ip_df.to_csv('data/ip/dip_docs.csv',index=False)
ip_df= pd.read_csv('data/ip/dip_docs.csv')
ip_df.head()

,lev1,lev2,lev3,lev1_url,lev2_url,lev3_url
0,กฎหมาย - การป้องปรามการละเมิดทรัพย์สินทางปัญญา,NaN,กฎหมายที่เกี่ยวกับการป้องปรามการละเมิดทรัพย์สินทางปัญญา,https://www.ipthailand.go.th/th/dip-law-2/category/กฎหมาย-ประกาศ-ระเบียบ-คำสั่ง-การป้องปรามการละเมิดทรัพย์สินทางปัญญา.html,NaN,https://www.ipthailand.go.th/th/protect-002.html
1,กฎหมาย - การผลิตผลิตภัณฑ์ซีดี,กฏกระทรวง (Ministerial regulations),กฎกระทรวงว่าด้วยการกำหนดผลิตภัณฑ์ซีดีและอุปกรณ์ของเครื่องจักร พ.ศ. 2548 (ลงวันที่ 24 สิงหาคม 2548),https://www.ipthailand.go.th/th/cd-008.html,https://www.ipthailand.go.th/th/dip-law-2/category/กฏกระทรวง-ministerial-regulations.html,https://www.ipthailand.go.th/th/dip-law-2/item/กฎกระทรวง-ว่าด้วยการกำหนดผลิตภัณฑ์ซีดีและอุปกรณ์ของเครื่องจักร-พ-ศ-๒๕๔๘.html
2,กฎหมาย - การผลิตผลิตภัณฑ์ซีดี,คำสั่งกรมทรัพย์สินทางปัญญา (Orders of DIP),คำสั่งกรมทรัพย์สินทางปัญญา ที่ 152/2548 เรื่อง มอบอำนาจให้ผู้ว่าราชการจังหวัดปฏิบัติราชการแทน (เพิ่มเติม) (ลงวันที่ 23 สิงหาคม 2548),https://www.ipthailand.go.th/th/cd-008.html,https://www.ipthailand.go.th/th/dip-law-2/category/คำสั่งกรมทรัพย์สินทางปัญญา-orders-of-dip-3.html,https://www.ipthailand.go.th/th/dip-law-2/item/คำสั่งกรม-ที่-152-2548-เรื่อง-มอบอำนาจให้ผู้ว่าราชการจังหวัดปฏิบัติราชการแทน-เพิ่มเติม.html
3,กฎหมาย - การผลิตผลิตภัณฑ์ซีดี,คำสั่งกรมทรัพย์สินทางปัญญา (Orders of DIP),คำสั่งกรมทรัพย์สินทางปัญญา ที่ 189/2553 เรื่อง มอบอำนาจให้ข้าราชการปฏิบัติราชการตามพระราชบัญญัติการผลิตผลิตภัณฑ์ซีดี พ.ศ. 2548 แทนอธิบดีกรมทรัพย์สินทางปัญญา (ลงวันที่ 28 เมษายน 2553),https://www.ipthailand.go.th/th/cd-008.html,https://www.ipthailand.go.th/th/dip-law-2/category/คำสั่งกรมทรัพย์สินทางปัญญา-orders-of-dip-3.html,https://www.ipthailand.go.th/th/dip-law-2/item/คำสั่งกรมทรัพย์สินทางปัญญาที่-๑๘๙-๒๕๕๓-เรื่อง-มอบอำนาจให้ข้าราชการปฏิบัติราชการตามพระราชบัญญัติการผล.html
4,กฎหมาย - การผลิตผลิตภัณฑ์ซีดี,คำสั่งกรมทรัพย์สินทางปัญญา (Orders of DIP),คำสั่งกรมทรัพย์สินทางปัญญา ที่ 229/2553 เรื่อง ให้ข้าราชการในสังกัดสำนักงานพาณิชย์จังหวัดปฏิบัติหน้าที่ราชการในฐานะพนักงานเจ้าหน้าที่และมีบัตรประจำตัวพนักงานเจ้าหน้าที่ตามพระราชบัญญัติการผลิตผลิตภัณฑ์ซีดี พ.ศ. 2548 (ลงวันที่ 14 พฤษภาคม 2553),https://www.ipthailand.go.th/th/cd-008.html,https://www.ipthailand.go.th/th/dip-law-2/category/คำสั่งกรมทรัพย์สินทางปัญญา-orders-of-dip-3.html,https://www.ipthailand.go.th/th/dip-law-2/item/คำสั่งกรมทรัพย์สินทางปัญญา-ที่-๒๒๙-๒๕๕๓-เรื่อง-ให้ข้าราชการในสังกัดสำนักงานพาณิชย์จังหวัดปฏิบัติหน้า.html


In [580]:
v02.merge(ip_df,left_on='title',right_on='lev3').drop('txt',1)

,sysid,title,law_type,lev1,lev2,lev3,lev1_url,lev2_url,lev3_url
0,454734,พระราชบัญญัติการผลิตผลิตภัณฑ์ซีดี พ.ศ. 2548,พระราชบัญญัติ,กฎหมาย - การผลิตผลิตภัณฑ์ซีดี,พระราชบัญญัติ (Acts),พระราชบัญญัติการผลิตผลิตภัณฑ์ซีดี พ.ศ. 2548,https://www.ipthailand.go.th/th/cd-008.html,https://www.ipthailand.go.th/th/dip-law-2/category/พระราชบัญญัติ-acts-5.html,https://www.ipthailand.go.th/th/dip-law-2/item/พระราชบัญญัติการผลิตผลิตภัณฑ์-ซีดี-พ-ศ-2548.html
1,328557,พระราชบัญญัติคุ้มครองแบบผังภูมิของวงจรรวม พ.ศ. 2543,พระราชบัญญัติ,กฎหมาย - แบบผังภูมิของวงจรรวม,พระราชบัญญัติ (Acts),พระราชบัญญัติคุ้มครองแบบผังภูมิของวงจรรวม พ.ศ. 2543,https://www.ipthailand.go.th/th/layout-007.html,https://www.ipthailand.go.th/th/dip-law-2/category/พระราชบัญญัติ-acts-6.html,https://www.ipthailand.go.th/th/dip-law-2/item/พระราชบัญญัติแบบผังภูมิของวงจรรวม-พ-ศ-2543.html
2,797397,ประกาศกรมทรัพย์สินทางปัญญา เรื่อง กำหนดค่าดำเนินการในต่างประเทศภายใต้พิธีสารมาดริด,ประกาศ,กฎหมาย - เครื่องหมายการค้า,ประกาศกรมทรัพย์สินทางปัญญา (Notifications of DIP),ประกาศกรมทรัพย์สินทางปัญญา เรื่อง กำหนดค่าดำเนินการในต่างประเทศภายใต้พิธีสารมาดริด,https://www.ipthailand.go.th/th/trademark-008.html,https://www.ipthailand.go.th/th/dip-law-2/category/ประกาศกรมทรัพย์สินทางปัญญา-notifications-of-dip-2.html,https://www.ipthailand.go.th/th/dip-law-2/item/ประกาศกรมทรัพย์สินทางปัญญา-เรื่อง-กำหนดค่าดำเนินการในต่างประเทศภายใต้พิธีสารมาดริด.html
3,797395,ประกาศกรมทรัพย์สินทางปัญญา เรื่อง กำหนดแบบพิมพ์หนังสือแจ้ง หนังสือเรียก และหนังสืออื่นใดภายใต้พิธีสารมาดริด,ประกาศ,กฎหมาย - เครื่องหมายการค้า,ประกาศกรมทรัพย์สินทางปัญญา (Notifications of DIP),ประกาศกรมทรัพย์สินทางปัญญา เรื่อง กำหนดแบบพิมพ์หนังสือแจ้ง หนังสือเรียก และหนังสืออื่นใดภายใต้พิธีสารมาดริด,https://www.ipthailand.go.th/th/trademark-008.html,https://www.ipthailand.go.th/th/dip-law-2/category/ประกาศกรมทรัพย์สินทางปัญญา-notifications-of-dip-2.html,https://www.ipthailand.go.th/th/dip-law-2/item/ประกาศกรมทรัพย์สินทางปัญญา-เรื่อง-กำหนดแบบพิมพ์หนังสือแจ้ง-หนังสือเรียก-และหนังสืออื่นใดภายใต้พิธีสารมาดริด.html
4,855436,ประกาศกรมทรัพย์สินทางปัญญา เรื่อง กำหนดแบบพิมพ์และวิธีการยื่นคำขอจดทะเบียนและคำขออื่นๆ ภายใต้พิธีสารมาดริด,ประกาศ,กฎหมาย - เครื่องหมายการค้า,ประกาศกรมทรัพย์สินทางปัญญา (Notifications of DIP),ประกาศกรมทรัพย์สินทางปัญญา เรื่อง กำหนดแบบพิมพ์และวิธีการยื่นคำขอจดทะเบียนและคำขออื่นๆ ภายใต้พิธีสารมาดริด,https://www.ipthailand.go.th/th/trademark-008.html,https://www.ipthailand.go.th/th/dip-law-2/category/ประกาศกรมทรัพย์สินทางปัญญา-notifications-of-dip-2.html,https://www.ipthailand.go.th/th/dip-law-2/item/ประกาศกรมทรัพย์สินทางปัญญา-เรื่อง-กำหนดแบบพิมพ์และวิธีการยื่นคำขอจดทะเบียนและคำขออื่นๆ-ภายใต้พิธีสารมาดริด.html
5,826636,คำสั่งหัวหน้าคณะรักษาความสงบแห่งชาติ ที่ 1/2562 เรื่อง การแก้ไขเพิ่มเติมกฎหมายว่าด้วยสิทธิบัตรและมาตรการด้านสิทธิบัตรเป็นกรณีพิเศษ,คำสั่ง,กฎหมาย - สิทธิบัตร,คำสั่งหัวหน้าคณะรักษาความสงบแห่งชาติ,คำสั่งหัวหน้าคณะรักษาความสงบแห่งชาติ ที่ 1/2562 เรื่อง การแก้ไขเพิ่มเติมกฎหมายว่าด้วยสิทธิบัตรและมาตรการด้านสิทธิบัตรเป็นกรณีพิเศษ,https://www.ipthailand.go.th/th/patent-008.html,https://www.ipthailand.go.th/th/dip-law-2/category/คำสั่งหัวหน้าคณะรักษาความสงบแห่งชาติ.html,https://www.ipthailand.go.th/th/dip-law-2/item/คำสั่งหัวหน้าคณะรักษาความสงบแห่งชาติ-ที่-1-2562-เรื่อง-การแก้ไขเพิ่มเติมกฎหมายว่าด้วยสิทธิบัตรและมาตรการด้านสิทธิบัตรเป็นกรณีพิเศษ.html
6,826778,คำสั่งหัวหน้าคณะรักษาความสงบแห่งชาติ ที่ 1/2562 เรื่อง การแก้ไขเพิ่มเติมกฎหมายว่าด้วยสิทธิบัตรและมาตรการด้านสิทธิบัตรเป็นกรณีพิเศษ,คำสั่ง,กฎหมาย - สิทธิบัตร,คำสั่งหัวหน้าคณะรักษาความสงบแห่งชาติ,คำสั่งหัวหน้าคณะรักษาความสงบแห่งชาติ ที่ 1/2562 เรื่อง การแก้ไขเพิ่มเติมกฎหมายว่าด้วยสิทธิบัตรและมาตรการด้านสิทธิบัตรเป็นกรณีพิเศษ,https://www.ipthailand.go.th/th/patent-008.html,https://www.ipthailand.go.th/th/dip-law-2/category/คำสั่งหัวหน้าคณะรักษาความสงบแห่งชาติ.html,https://www.ipthailand.go.th/th/dip-law-2/item/คำสั่งหัวหน้าคณะรักษาความสงบแห่งชาติ-ที่-1-2562-เรื่อง-การแก้ไขเพิ่มเติมกฎหมายว่าด้วยสิทธิบัตรและมาตรการด้านสิทธิบัตรเป็นกรณีพิเศษ.html
